In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [9]:
np.random.seed(1967)

def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
        
    Returns:
        - (matrix): The associated matrix to these parameters.
    """
    alpha, beta, gamma, phi = params

    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])

    # Calculate the individual rotation matrices
    rz_gamma = np.array([[np.exp(-1j * gamma / 2), 0], [0, np.exp(1j * gamma / 2)]])

    rx_beta = np.array([[np.cos(beta / 2), -1j * np.sin(beta / 2)],
                        [-1j * np.sin(beta / 2), np.cos(beta / 2)]])
    
    rz_alpha = np.array([[np.exp(-1j * alpha / 2), 0], [0, np.exp(1j * alpha / 2)]])

    # Combine the rotation matrices in the specified order
    unitary_matrix = np.exp(1j * phi) * rz_gamma @ rx_beta @ rz_alpha

    return unitary_matrix

    # Put your code here #

    # Return the matrix

def error(U, params):
    """
    This function determines the similarity between your generated matrix and
    the target unitary.

    Args:
        - U (np.array): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)
    
    # Put your code here #
    error_value = np.linalg.norm(U - matrix)

    # Return the error
    return error_value


In [12]:
def train_parameters(U):
    epochs = 1000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        params -= lr * grad(U, params)

    return params

In [15]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    return json.dumps(params)


def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    
    print("matrix1:", matrix1)
    print("matrix2:", matrix2)
    
    assert np.allclose(matrix1, matrix2, atol=0.2)


# These are the public test cases
test_cases = [
    ('[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'),
    ('[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")
        
    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
matrix1: [[ 0.34254857-0.5957681j   0.36292691-0.6292892j ]
 [ 0.36318419-0.62914075j -0.34436144+0.59472208j]]
matrix2: [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]


AssertionError: 